In [74]:
import urllib.request
import json
import pymongo
import time
import numpy as np

def get_accomidation_details(number):
    base_url = "https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip={}"
    acc_url = base_url.format(number)
    
    try:
        with urllib.request.urlopen(acc_url) as response:
            
            return json.load(response)
    except urllib.error.URLError as e:
        print("The API URL is invalid."+e)
    except json.JSONDecodeError:
        print("Error decoding JSON response")

    

In [ ]:
dat = np.arange(0, 2200, 50)
for dat in np.arange(0,2400,50):
    print(dat)
    print(get_accomidation_details(dat))
    time.sleep(10)

0
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=50', 'results': [{'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'Taravie Hotel', 'url': 'https://www.facebook.com/pages/Taravie-Hotel/200397549970664', 'geo': {'@type': 'GeoCoordinates', 'longitude': -6.22803760615159, 'latitude': 52.6453261952866}, 'telephone': '+353(0)539425208', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Courtown', 'addressRegion': 'Wexford', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'Hotel', 'Follow the Shamrock', 'Standard']}, {'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'Adare Manor Hotel & Golf Resort', 'url': 'https://www.adaremanor.com/', 'geo': {'@type': 'GeoCoo

50
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=100', 'results': [{'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Ceecliff House', 'url': 'http://www.bandbireland.com/accommodation/28139/ceecliff-house', 'geo': {'@type': 'GeoCoordinates', 'longitude': -7.16756763252351, 'latitude': 55.2892431392363}, 'telephone': '+353(0)749379159', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Culdaff', 'addressRegion': 'Donegal', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'B&B', 'B&B Ireland']}, {'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'The Woodford Dolmen Hotel', 'url': 'http://www.woodforddolmenhotel.ie', 'geo': {'@type': 'GeoCoordinates

100
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=150', 'results': [{'@context': 'http://schema.org', '@type': ['LodgingBusiness', 'LocalBusiness'], 'name': 'Elizabeth Fort', 'url': 'http://irishlandmark.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -8.47758645938757, 'latitude': 51.8946773903946}, 'telephone': '+353(0)16704733', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Cork City', 'addressRegion': 'Cork', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'Self Catering Accommodation', 'Welcome Standard']}, {'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Sancta Maria', 'url': 'http://sanctamariabb.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -9.489

150
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=200', 'results': [{'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Forest Park House', 'url': 'https://bedandbreakfastboyle.com/', 'geo': {'@type': 'GeoCoordinates', 'longitude': -8.244966, 'latitude': 53.969937}, 'telephone': '+353(0)719662227', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Boyle', 'addressRegion': 'Roscommon', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'B&B', 'Follow the Shamrock', 'Standard', "Ireland's Hidden Hearthlands"]}, {'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Ringfort@The Irish National Heritage Park', 'url': 'http://www.irishheritage.ie', 

200
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=250', 'results': [{'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Old Cable Historic House', 'url': 'http://www.oldcablehouse.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -10.172036, 'latitude': 51.831558}, 'telephone': '+353(0)669474233', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Waterville', 'addressRegion': 'Kerry', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'B&B', 'Follow the Shamrock', 'Standard', 'B&Bs', 'Historic House']}, {'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Elderberry', 'url': 'https://www.bandbireland.com/accommodation/28063/elderberry-b-b

...

1000
Error decoding JSON response
None
1050
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=1100', 'results': [{'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Riverside Cottage', 'url': 'http://www.riversidecottagedoolin.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -9.38345055872196, 'latitude': 53.0131782342571}, 'telephone': '+353(0)657074049', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Doolin', 'addressRegion': 'Clare', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'B&B', 'Follow the Shamrock', 'Standard']}, {'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': "Michaeleen's Manor B&B", 'url': 'http://www.michaeleensma

1100
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=1150', 'results': [{'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'Barberstown Castle', 'url': 'http://www.barberstowncastle.ie', 'geo': {'@type': 'GeoCoordinates', 'longitude': -6.597387, 'latitude': 53.153086}, 'telephone': '+353(0)16288157', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Straffan', 'addressRegion': 'Kildare', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'Hotel', 'Follow the Shamrock', 'Standard']}, {'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'Sandymount Hotel', 'url': 'http://www.sandymounthotel.ie', 'geo': {'@type': 'GeoCoordinates', 'longitude': -6.224431, 'latitude': 5

1150
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=1200', 'results': [{'@context': 'http://schema.org', '@type': ['Campground', 'LocalBusiness'], 'name': 'Renvyle Beach Caravan and Camping Park', 'url': 'http://www.renvylebeachcaravanpark.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -9.984855, 'latitude': 53.602784}, 'telephone': '+3539543462', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Renvyle', 'addressRegion': 'Galway', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'Camping', 'Follow the Shamrock', 'Standard']}, {'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Delphi Lodge', 'url': 'http://www.delphilodge.ie', 'geo': {'@type': 'GeoCoordinates', 'longit

1200
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=1250', 'results': [{'@context': 'http://schema.org', '@type': ['BedAndBreakfast', 'LocalBusiness'], 'name': 'Macreddin Rock B&B', 'url': 'http://www.macreddinrock.com', 'geo': {'@type': 'GeoCoordinates', 'longitude': -6.331155, 'latitude': 52.884021}, 'telephone': '+353(0)40236369', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Aughrim', 'addressRegion': 'Wicklow', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'B&B', 'Follow the Shamrock', 'Standard']}, {'@context': 'http://schema.org', '@type': ['LodgingBusiness', 'LocalBusiness'], 'name': 'Silver Crescent Coach House & Apartments', 'url': 'https://www.yourtimeinireland.ie', 'geo': {'@type': 'GeoCoordinate

1250
{'count': '2396', 'nextPage': 'https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip=1300', 'results': [{'@context': 'http://schema.org', '@type': ['Hostel', 'LocalBusiness'], 'name': 'Moville Boutique Hostel Accommodation', 'url': 'http://www.facebook.com/moville.hostel/timeline?ref=page_internal', 'geo': {'@type': 'GeoCoordinates', 'longitude': -7.04598988863495, 'latitude': 55.1913552178344}, 'telephone': '+353(0)749382378', 'address': {'@type': 'PostalAddress', 'addressLocality': 'Inishowen', 'addressRegion': 'Donegal', 'addressCountry': 'Republic of Ireland'}, 'image': {'@type': 'ImageObject', 'caption': 'Fáilte Ireland Logo', 'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'}, 'tags': ['Accommodation', 'Hostel', 'Welcome Standard']}, {'@context': 'http://schema.org', '@type': ['Hotel', 'LocalBusiness'], 'name': 'County Arms Hotel', 'url': 'http://www.countyarmshotel.com', 'geo': {'@typ

In [59]:
from pymongo import MongoClient

ACCOM_DATA = get_accomidation()

client = pymongo.MongoClient("mongodb://localhost:27017/",username="dap",password="dap")
db = client['test_project']
collection = db['project-collection']

print(len(ACCOM_DATA["results"]))


50


In [60]:
for i in range (0 , len(ACCOM_DATA["results"])):
    collection.insert_one(ACCOM_DATA["results"][i])

In [61]:
import pprint
for collections in collection.find():
    pprint.pprint(collections)

{'@context': 'http://schema.org',
 '@type': ['Hotel', 'LocalBusiness'],
 '_id': ObjectId('6424a94001b97d05e9144478'),
 'address': {'@type': 'PostalAddress',
             'addressCountry': 'Republic of Ireland',
             'addressLocality': 'Courtown',
             'addressRegion': 'Wexford'},
 'geo': {'@type': 'GeoCoordinates',
         'latitude': 52.6453261952866,
         'longitude': -6.22803760615159},
 'image': {'@type': 'ImageObject',
           'caption': 'Fáilte Ireland Logo',
           'url': 'https://failtecdn.azureedge.net/failteireland/F%C3%A1ilte_Ireland_Logo_OpenDataAPI.jpg'},
 'name': 'Taravie Hotel',
 'tags': ['Accommodation', 'Hotel', 'Follow the Shamrock', 'Standard'],
 'telephone': '+353(0)539425208',
 'url': 'https://www.facebook.com/pages/Taravie-Hotel/200397549970664'}
{'@context': 'http://schema.org',
 '@type': ['Hotel', 'LocalBusiness'],
 '_id': ObjectId('6424a94001b97d05e9144479'),
 'address': {'@type': 'PostalAddress',
             'addressCountry': 'Repu

In [ ]:
def get_accomidation_details(number):
    try:
        with req.urlopen("https://failteireland.azure-api.net/opendata-api/v1/accommodation?subscription-key=&search=*&$skip={}").format(number) as response:
        return json.load(response)
    except urllib.error.URLError:
        print("The URL could not be opened")
    else:
        print("The URL opened correctly")